In [1]:
import numpy as np
from bayesian_optimization import BayesianOptimization, BayesianOptimization1dDemo

In [2]:
def f(x):
    return -(3.671 * x - 1.956)**2 * np.sin(9.5 * x - 3.6)

In [3]:
np.random.seed(98765)
bo = BayesianOptimization1dDemo(f, np.array([[0.0, 1.0]]), policy='ei')
X, y = bo.fit()

In [ ]:
# convert -delay 100 -loop 0 -dispose previous ei_iteration_0*.png ei.gif

In [4]:
from IPython.display import HTML
HTML('<img src="ei.gif">')

In [5]:
np.random.seed(98765)
bo = BayesianOptimization1dDemo(f, np.array([[0.0, 1.0]]), policy='ucb')
X, y = bo.fit()

In [ ]:
# convert -delay 100 -loop 0 -dispose previous ucb_iteration_0*.png ucb.gif

In [6]:
from IPython.display import HTML
HTML('<img src="ucb.gif">')